## Basic exploration

### Import modules

In [ ]:
import os, sys, time, random, math
from datetime import datetime      # For timer 
import tarfile, zipfile            # Work with compressed files

import numpy as np                 # Linear algebra
import pandas as pd                # Data processing

from IPython.display import display, Image  # Nice print statements
from ggplot import *               # yhat/ggplot for plots

from subprocess import check_output
print(check_output(["ls", "../../input"]).decode("utf8"))

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        tmin, tsec = divmod((datetime.now() - start_time).total_seconds(), 60)
        print(' Time taken: %i minutes and %s seconds.' %
              (tmin, round(tsec, 2)))

### Load data

In [ ]:
train_data = pd.read_csv("../../input/train.csv")
test_data = pd.read_csv("../../input/test.csv")

start_time = timer(None)
ntrain = train_data.shape[0]
ntest = test_data.shape[0]
train_test = pd.concat((train_data[features], test_data[features])).reset_index(drop=True)
for c in range(len(cat_features)):
    train_test[cat_features[c]] = train_test[cat_features[c]].astype('category').cat.codes

X = train_test.iloc[:ntrain,:]
X_test = train_test.iloc[ntrain:,:]
y = np.log(train_data['loss'])

timer(start_time)


### Test with lightGBM
LightGBM wants data in a CSV file. Label as first column. No headers.

In [ ]:
lgbm_train = pd.concat([y, X], axis = 1)
display( lgbm_train.head())
display( lgbm_train.shape)
# lgbm_train.to_csv("lgbm_train.csv", index = None)
# X_test.to_csv("lgbm_test.csv", index=None)
# print('LightGBM files saved.')

After LightGBM is run externally:
- Get results from LightGBM in LighGBM_predict_result.txt
- Read thm into pd.df
- Attach id's
- Save into csv for submission.

In [ ]:
submission = pd.read_csv("LightGBM_predict_result.txt", header=None)
display(submission.shape)
display(submission.head())
submission = pd.concat([test_data['id'], np.exp(submission)], axis = 1, names = ['id', 'loss'])
submission.columns = ['id', 'loss']
display(submission.shape)
display(submission.head())
submission.to_csv("submission_lightgbm_L2onLog_7234_trees.csv", index=None)